import torch
import torch.nn as nn

In [104]:
bn = nn.BatchNorm2d(3, track_running_stats=True)  # momentum=0.1是默认的
print(bn)
print(bn.running_mean)
print(bn.running_var)

x = torch.randn(4, 3, 2, 2)
y = bn(x)
# print(x)
# print(y)
print(y.size())
print(bn)
print(bn.running_mean)
print(bn.running_var)

BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
tensor([0., 0., 0.])
tensor([1., 1., 1.])
torch.Size([4, 3, 2, 2])
BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
tensor([0.0151, 0.0008, 0.0269])
tensor([0.9738, 1.0608, 1.0294])


In [103]:
a = (x[0, 0, :, :] + x[1, 0, :, :] + x[2, 0, :, :] + x[3, 0, :, :]).sum() / 16
b = (x[0, 1, :, :] + x[1, 1, :, :] + x[2, 1, :, :] + x[3, 1, :, :]).sum() / 16
c = (x[0, 2, :, :] + x[1, 2, :, :] + x[2, 2, :, :] + x[3, 2, :, :]).sum() / 16
print('The mean value of the first channel is %f' % a.data)
print('The mean value of the second channel is %f' % b.data)
print('The mean value of the third channel is %f' % c.data)
print('The output mean value of the BN layer is %f, %f, %f' % (bn.running_mean.data[0], bn.running_mean.data[1], bn.running_mean.data[2]))
print('The output var value of the BN layer is %f, %f, %f' % (bn.running_var.data[0], bn.running_var.data[1], bn.running_var.data[2]))

The mean value of the first channel is -0.486114
The mean value of the second channel is -0.177291
The mean value of the third channel is 0.145817
The output mean value of the BN layer is -0.048611, -0.017729, 0.014582
The output var value of the BN layer is 0.981166, 0.958100, 0.996654


In [7]:
"""
第一个通道的running_mean正好是真实的mean的0.1倍. 同理running_var
因为在最开始的时候running_mean=0，然后用滑动平均公式去更新：
running_mean=0.9∗running_mean+0.1∗mean=0.1∗mean
"""

'\n第一个通道的running_mean正好是真实的mean的0.1倍\n因为在最开始的时候running_mean=0，然后用滑动平均公式去更新：\nrunning_mean=0.9∗running_mean+0.1∗mean=0.1∗mean\n'

In [105]:
# 那么当我们前向传播两次之后，再来观察running_mean和mean的关系。
# running_mean=0.9∗(0.9∗0+0.1∗mean)+0.1∗mean=0.19∗mean
bn = nn.BatchNorm2d(3)
x = torch.randn(4, 3, 2, 2)
y = bn(x)
y = bn(x)  # 前向传播两次
a = (x[0, 0, :, :] + x[1, 0, :, :] + x[2, 0, :, :] + x[3, 0, :, :]).sum() / 16
b = (x[0, 1, :, :] + x[1, 1, :, :] + x[2, 1, :, :] + x[3, 1, :, :]).sum() / 16
c = (x[0, 2, :, :] + x[1, 2, :, :] + x[2, 2, :, :] + x[3, 2, :, :]).sum() / 16
print('The mean value of the first channel is %f' % a.data)
print('The mean value of the first channel is %f' % b.data)
print('The mean value of the first channel is %f' % c.data)
print('The output mean value of the BN layer is %f, %f, %f' % (bn.running_mean.data[0], bn.running_mean.data[1], bn.running_mean.data[2]))
print("running_mean:", bn.running_mean)
print("running_var:", bn.running_var)

The mean value of the first channel is 0.085720
The mean value of the first channel is 0.178778
The mean value of the first channel is 0.379047
The output mean value of the BN layer is 0.016287, 0.033968, 0.072019
running_mean: tensor([0.0163, 0.0340, 0.0720])
running_var: tensor([1.1124, 0.8708, 0.9671])


In [82]:
import numpy, torch
a = numpy.array([[1, 2, 4, 1],[6, 3, 2, 4],[2, 4, 6, 1]])
# print(a.size())
print("mean:", a.mean(0))
print("ddof = 0 -> 有样本方差/标准差。|| ddof = 1 -> 无样本方差/标准差。")
print("variance(1):", a.var(0, ddof=1))
print("std(1):", a.std(0, ddof=1))
print("variance(0):", a.var(0, ddof=0))
print("std(0):", a.std(0, ddof=0))

print("\n" + "========="*5)
print("有偏和无偏的区别在于无偏的分母是N-1，有偏的分母是N。注意在BatchNorm中，用于更新running_var时，使用无偏样本方差即，但是在对batch进行归一化时，使用有偏样本方差，因此如果batch_size=1，会报错。")
print("========="*5 + "\n")

a = torch.tensor([[1., 2., 4., 1.],[6., 3., 2., 4.],[2., 4., 6., 1.]])

bn = torch.nn.BatchNorm2d(4) # 注意torch都是要操作4D的数据的
print("初始化的running_mean,shape取决于你的chs:", bn.running_mean)
print("初始化的running_var,shape取决于你的chs:", bn.running_var)

mean: [3. 3. 4. 2.]
ddof = 0 -> 有样本方差/标准差。|| ddof = 1 -> 无样本方差/标准差。
variance(1): [7. 1. 4. 3.]
std(1): [2.64575131 1.         2.         1.73205081]
variance(0): [4.66666667 0.66666667 2.66666667 2.        ]
std(0): [2.1602469  0.81649658 1.63299316 1.41421356]

有偏和无偏的区别在于无偏的分母是N-1，有偏的分母是N。注意在BatchNorm中，用于更新running_var时，使用无偏样本方差即，但是在对batch进行归一化时，使用有偏样本方差，因此如果batch_size=1，会报错。

初始化的running_mean,shape取决于你的chs: tensor([0., 0., 0., 0.])
初始化的running_var,shape取决于你的chs: tensor([1., 1., 1., 1.])
torch.Size([1, 1, 3, 4])


RuntimeError: running_mean should contain 1 elements not 4